<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning Para Aplicações Biomédicas</font>
## <font color='blue'>Projeto 5 - Parte 1</font>
## <font color='blue'>Machine Learning Para Prever a Resistência a Antibióticos</font>

## Pacotes Python Usados no Projeto

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
# Imports
import pandas as pd
import numpy as np
from functools import reduce

In [3]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Compreendendo os Datasets de Origem

Veja a definição dos datasets no videobook do Capítulo 11 do curso.

## Carregando o Dataset de Metadados

In [4]:
# Carrega os dados
dataset_1_metadados = pd.read_csv('dataset1.csv')

In [5]:
# Shape
dataset_1_metadados.shape

(1936, 14)

In [6]:
# Amostra de dados
dataset_1_metadados.sample(10)

,Isolate,Year,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,TBM,TMP,CIP
534,11658_5#69,2004.0,S,S,NaN,S,S,S,S,NaN,S,NaN,NaN,R
1121,11679_7#20,2012.0,S,S,S,NaN,S,S,S,S,S,S,R,R
1832,24742_1#277,2017.0,S,S,R,NaN,NaN,NaN,S,S,S,S,S,S
1428,11791_6#7,2009.0,S,S,R,NaN,R,S,R,S,S,R,R,R
1835,24742_1#280,2017.0,R,S,R,NaN,NaN,NaN,R,R,S,S,S,S
248,11657_7#63,2007.0,S,S,NaN,R,S,S,S,NaN,S,NaN,NaN,S
787,11658_8#58,2001.0,S,NaN,NaN,S,S,S,S,NaN,S,NaN,NaN,S
1211,11679_8#19,2002.0,S,S,NaN,S,S,S,S,NaN,S,NaN,NaN,S
1919,24742_1#58,2008.0,R,R,R,NaN,NaN,NaN,R,R,R,R,R,S
1403,11791_6#1,2007.0,S,S,R,NaN,S,S,S,S,R,S,S,S


- **Número do Isolado (Isolate)**: Número exclusivo para identificar uma cepa específica de bactérias *E. coli*. Portanto, nos referiremos a cada linha do nosso conjunto de dados como um “isolado” de agora em diante.

- **Ano de Isolamento (Year)**: O ano em que uma determinada cepa de bactéria foi separada de seu ambiente natural, daí o nome *isolado*.

- **Antibióticos**: Existem **12 colunas de antibióticos** nomeadas de acordo com sua **abreviatura de 3 letras** adotada pela "Sociedade Britânica de Quimioterapia Antimicrobiana". Página: https://bsac.org.uk/



|Abreviação|Classe: Subclasse|Nome|
|:----------|:--------------|:--------|
|**CTZ**|Beta-lactams: Cephalosporins|Ceftazidime|
|**CTX**|Beta-lactams: Cephalosporins|Cefotaxime |
|**CXM**|Beta-lactams: Cephalosporins|Cefuroxime|
|**CET**|Beta-lactams: Cephalosporins|Cephalothin|
|**AMP**|Beta-lactams: Penicillin|Ampicillin|
|**AMX**|Beta-lactams: Penicillin|Amoxicillin|
|**AMC**|Beta-lactams: Penicillin|Amoxicillin + Clavulanate potassium|
|**TZP**|Beta-lactams: Piperacillin|Tazobactam|
|**GEN**|Aminoglycosides|Gentamicin|
|**TBM**|Aminoglycosides|Tobramycin|
|**TMP**|Antifolate|Trimethoprim|
|**CIP**|Fluoroquinolones|Ciprofloxacin|

**Determinação dos Valores Resistente (R) e Suscetível (S):**

**Teste de suscetibilidade antimicrobiana**: São testes laboratoriais, onde concentrações específicas de um medicamento são testadas para cada isolado de bactéria para determinar se ele é **Resistente (R)**, **Suscetível (S)** ou **Intermediário (I)**.

- **Pontos de interrupção clínicos**: Cada um dos medicamentos tem uma concentração diferente (ponto de interrupção clínico) usada para determinar a resistência ou suscetibilidade de *E. coli* a esse medicamento. Estas concentrações são acordadas pelas agências de saúde pública com base em diferentes fatores e podem variar de ano para ano, dadas as novas descobertas. Eles servem como uma ferramenta de orientação para os médicos prescreverem um antibiótico específico e uma dose apropriada para os pacientes.

- **Os resultados de resistência (R) e suscetível (S)** dos testes laboratoriais foram determinados com base na série de diretrizes do [Comitê Europeu de Testes de Suscetibilidade Antimicrobiana (EUCAST)](https://www.eucast.org/videos_and_online_seminars/english) em 25/01/2017. Para este estudo, os isolados classificados como **Intermediários (I)** foram agrupados com os **Resistentes (R)**.

**Nota**: NaN foi usado para marcar dados faltantes, para aqueles isolados que não foram testados com um medicamento específico.

## Carregando o Dataset de Presença e Ausência de Genes

In [7]:
# Carrega os dados
dataset_2_genes = pd.read_csv('dataset2.csv')

In [8]:
# Shape
dataset_2_genes.shape

(2033, 17199)

In [9]:
# Amostra
dataset_2_genes.sample(10)

,Unnamed: 0,yeiU,yhhS,ybaE,eutR,ibrB,ytfP,aslB,narQ,tolR,...,group_48768,group_48873,group_48916,group_48933,group_48937,group_48958,group_49020,group_49174,group_49253,group_49257
45,11657_5#51,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1937,24742_1#348,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1140,11679_7#4,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
763,11658_8#34,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1077,11679_6#67,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
600,11658_6#40,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
827,11679_4#10,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
896,11679_4#74,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1507,12045_3#66,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1401,11791_6#17,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


**Genes:** São pedaços de DNA que podem ter tamanhos diferentes. Alguns genes codificam proteínas específicas e sua função é bem compreendida, enquanto a função de outros genes ainda não é bem compreendida. Os genes podem ser geralmente classificados em:

- **Genes Centrais:** Estes são os genes que estão presentes em quase todos os indivíduos para uma espécie bacteriana específica. No nosso caso, são os genes que todos os nossos isolados de *E.coli* têm em comum.

- **Genes Acessórios:** São os genes que podem ser encontrados em um indivíduo, mas não em outro indivíduo da mesma espécie. Esses seriam os genes exclusivos de cada um de nossos isoaldos *E. coli*.

- **Pan-genoma:** Estes são todos os genes possíveis que podem ser encontrados em uma determinada espécie. Ou seja, são todos os genes presentes em nosso isolado *E. coli*.

Consideramos:

- **0** = Ausência do gene
- **1** = Presença do gene.

Como mencionado acima, nem todos os genes têm funções conhecidas e são nomeados, por isso o autor do dataset os separou em 2 conjuntos de genes que foram nomeados de forma diferente:

**1) Regiões Codificantes Conhecidas**: Que foram extraídas das sequências de DNA anotadas e nomeadas de acordo com a proteína que codificam.

No resultado acima os genes codificadores **não contêm a palavra "group_"** no título da coluna.

**2) Regiões Codificantes Desconhecidas**: sequências de DNA que não codificam uma proteína específica, mas que são agrupadas com base em **grupos de genes ortólogos**. Isto significa que estas são sequências onde não sabemos necessariamente a sua função, mas sabemos que elas existem em muitos dos isolados *E. coli*. **Eles recebem um nome que começa com "group_"**.

**Genes Homólogos**: Derivam do mesmo gene ancestral e podem ser:

- **Genes Ortólogos**: São sequências onde não necessariamente sabemos sua função, mas sabemos que elas existem em muitos dos isolados *E. coli*. Para nossos propósitos, estes são todos os isolados *E. coli* que pertencem a um grupo específico.

- **Genes Parálogos**: São sequências geradas através da duplicação de genes no genoma da mesma espécie e são responsáveis por versões alternativas de um determinado gene.

In [10]:
# Ajusta o nome da coluna
dataset_2_genes = dataset_2_genes.rename(columns = {'Unnamed: 0' : 'Isolate'})

In [11]:
# Amostra
dataset_2_genes.sample(10)

,Isolate,yeiU,yhhS,ybaE,eutR,ibrB,ytfP,aslB,narQ,tolR,...,group_48768,group_48873,group_48916,group_48933,group_48937,group_48958,group_49020,group_49174,group_49253,group_49257
1639,18090_7#87,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
761,11658_8#32,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1004,11679_5#85,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1513,18090_6#37,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
67,11657_5#71,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1553,18090_6#93,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
68,11657_5#72,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,11657_5#12,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
914,11679_4#90,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
848,11679_4#3,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Carregando o Dataset de Estrutura Populacional

In [12]:
# Carrega os dados
dataset_3_pop_struc = pd.read_csv('dataset3.csv')

In [13]:
# Shape
dataset_3_pop_struc.shape

(1936, 1072)

In [14]:
# Amostra
dataset_3_pop_struc.sample(10)

,Unnamed: 0,cutoff_2,cutoff_3,cutoff_4,cutoff_5,cutoff_6,cutoff_7,cutoff_8,cutoff_9,cutoff_10,...,cutoff_25459,cutoff_25654,cutoff_25772,cutoff_25979,cutoff_26792,cutoff_27119,cutoff_27236,cutoff_27248,cutoff_27690,cutoff_45092
495,11658_5#31,1373,1371,1364,1359,1358,1357,1354,1350,1339,...,0,0,0,0,0,0,0,0,0,0
1853,24742_1#303,947,944,939,933,932,931,929,926,916,...,0,0,0,0,0,0,0,0,0,0
523,11658_5#59,1405,1403,1396,1391,1390,1389,1386,1381,1370,...,0,0,0,0,0,0,0,0,0,0
1583,18090_7#35,647,646,643,640,640,639,637,633,624,...,0,0,0,0,0,0,0,0,0,0
577,11658_6#20,1464,1462,1455,1450,1449,1448,1445,1441,1430,...,0,0,0,0,0,0,0,0,0,0
1769,24742_1#211,853,851,847,842,841,840,838,835,825,...,0,0,0,0,0,0,0,0,0,0
1339,11791_3#49,377,376,374,372,372,371,370,366,359,...,0,0,0,0,0,0,0,0,0,0
688,11658_7#36,1587,1585,1578,1573,1572,1571,1568,1563,1551,...,0,0,0,0,0,0,0,0,0,0
1218,11679_8#25,244,243,241,239,239,238,237,233,228,...,0,0,0,0,0,0,0,0,0,0
351,11658_3#74,1214,1212,1207,1202,1201,1200,1197,1192,1182,...,0,0,0,0,0,0,0,0,0,0


**Estrutura populacional** é definida como a organização da variação genética em uma população ou espécie. É basicamente sobre quão semelhantes (geneticamente) são diferentes isolados. A ideia é que isolados geneticamente semelhantes talvez tenham o mesmo perfil de resistência.

**1) A coluna Isolate** agora é igual à coluna **Isolate** do arquivo csv de metadados.

**2) cutoff_#**: Os nomes deste conjunto de colunas representam os diferentes valores de corte (cutoff) usados para agrupar isolados em diferentes clusters. Por exemplo, o valor de corte de 3 (cutoff_3) significa que agrupamos em um cluster se os isolados tiverem 3 ou menos diferenças (também conhecidos como **SNPs**) entre eles. Ao final do processo cada isolado é classificado em um cluster diferente dependendo do valor de corte. Assim, cada valor de corte deve produzir um conjunto diferente de clusters. Os números abaixo de cada coluna são simplesmente o número de referência para cada cluster.

**NOTA:** **SNP (Polimorfismo de Nucleotídeo Único)** é uma única diferença em um nucleotídeo em um gene específico quando temos um par de isolados sendo comparados. Neste caso, cada isolado no conjunto de dados foi comparado para um **genoma de referência principal EC958**.

Se o valor de corte for menor, podemos esperar que a maioria dos isolados não será agrupada com nenhum outro isolado, formando um número maior de clusters, mas cada cluster seria composto por grupos de tamanhos menores. A soma de todos os membros de cada cluster deve produzir o número total de isolados (ou seja, 1936).

- Por exemplo, para **cutoff_2008** estamos considerando todos os isolados que possuem SNPs de 2008 ou menos. O código abaixo nos mostra que para este valor de corte temos um total de 517 clusters diferentes, cada um deles com um número diferente de isolados. Por exemplo, podemos ver que para o “cluster 222”, temos 221 isolados.

In [15]:
# Alterando a primeira coluna para corresponder aos outros dataframes
dataset_3_pop_struc = dataset_3_pop_struc.rename(columns = {'Unnamed: 0' : 'Isolate'})

In [16]:
# Filtrando o cutoff_2008
cutoff_2008 = pd.DataFrame(dataset_3_pop_struc['cutoff_2008'].value_counts())

In [17]:
cutoff_2008

,count
cutoff_2008,
222,221
111,208
334,75
134,57
472,45
...,...
154,1
153,1
151,1


Se o valor de corte for alto, podemos esperar que mais isolados sejam agrupados com outros isolados, formando um número menor de clusters, mas a maioria sendo agrupada em um cluster específico. Assim como antes, a soma de todos os membros de cada cluster deve ser igual ao nosso número total de isolados (ou seja, 1936).

- Por exemplo, se aumentarmos o valor de corte para **cutoff_27236**, ​​podemos observar que a maioria dos isolados (1932) cairia no "cluster 0" com apenas alguns isolados dispersos em outros clusters.

In [18]:
# Filtrando o cutoff_27236
cutoff_27236 = pd.DataFrame(dataset_3_pop_struc['cutoff_27236'].value_counts())

In [19]:
cutoff_27236

,count
cutoff_27236,
0,1932
1,1
2,1
3,1
4,1


Como não há informações sobre qual valor de corte nos forneceria a melhor maneira de agrupar nossos isolados ou quais membros do grupo estão necessariamente relacionados à resistência (R) ou à suscetibilidade (S), o autor do dataset incluiu 1.072 valores de corte diferentes, cada um com seus próprios grupos.

## Combinação dos Dados

Vamos criar um único dataset que será usado na etapa de modelagem.

In [20]:
# Os anos em que as cepas foram isoladas
lista_ano = dataset_1_metadados["Year"].unique()
lista_ano = lista_ano[np.logical_not(np.isnan(lista_ano))]

In [21]:
print(sorted(lista_ano))

[np.float64(1970.0), np.float64(1977.0), np.float64(1994.0), np.float64(1997.0), np.float64(1998.0), np.float64(1999.0), np.float64(2001.0), np.float64(2002.0), np.float64(2003.0), np.float64(2004.0), np.float64(2005.0), np.float64(2006.0), np.float64(2007.0), np.float64(2008.0), np.float64(2009.0), np.float64(2010.0), np.float64(2011.0), np.float64(2012.0), np.float64(2013.0), np.float64(2014.0), np.float64(2015.0), np.float64(2017.0)]


Vamos aplicar one-hot-encoding na coluna ano. O resultado será uma coluna para cada ano, com os valores:

- **"0"**: O isolamento foi realizado naquele ano.
- **"1"** : O isolamento não foi realizado naquele ano.

In [22]:
# Criando uma variável dummy com one-hot-encoding
dataset_1_metadados_dummies = pd.get_dummies(dataset_1_metadados, columns = ["Year"], dummy_na = False)

In [23]:
dataset_1_metadados_dummies

,Isolate,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,...,Year_2007.0,Year_2008.0,Year_2009.0,Year_2010.0,Year_2011.0,Year_2012.0,Year_2013.0,Year_2014.0,Year_2015.0,Year_2017.0
0,11657_5#10,S,S,S,NaN,S,S,S,S,S,...,False,False,False,True,False,False,False,False,False,False
1,11657_5#11,S,S,R,NaN,R,S,S,S,S,...,False,False,False,True,False,False,False,False,False,False
2,11657_5#12,S,S,S,NaN,S,S,S,S,S,...,False,False,False,True,False,False,False,False,False,False
3,11657_5#13,S,S,R,NaN,R,S,S,S,S,...,False,False,False,True,False,False,False,False,False,False
4,11657_5#14,S,S,R,NaN,S,S,S,S,S,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931,24742_1#96,S,S,S,NaN,NaN,NaN,S,S,S,...,False,False,False,False,False,False,False,False,False,False
1932,24742_1#97,S,S,S,NaN,NaN,NaN,S,S,S,...,False,False,False,False,False,False,False,False,False,False
1933,24742_1#98,S,S,R,NaN,NaN,NaN,S,S,S,...,False,False,False,False,False,False,False,False,False,False
1934,24742_1#99,S,S,R,NaN,NaN,NaN,S,S,S,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# Lista de todas as três fontes de dados
df_list = [dataset_1_metadados_dummies, dataset_2_genes, dataset_3_pop_struc]

In [25]:
# Criando um único dataframe com todos os medicamentos e recursos disponíveis
df_dsa = reduce(lambda left, right: pd.merge(left, right, on = ['Isolate'], how = 'inner'), df_list)

reduce é uma função da biblioteca functools que aplica uma função de forma cumulativa a todos os itens em uma lista, da esquerda para a direita, para reduzir a lista a um único valor.

Neste contexto, reduce está sendo usado para iterativamente unir (fazer merge) os DataFrames na lista df_list.

In [26]:
# Shape
df_dsa.shape

(1936, 18304)

In [27]:
# Amostra
df_dsa

,Isolate,CTZ,CTX,AMP,AMX,AMC,TZP,CXM,CET,GEN,...,cutoff_25459,cutoff_25654,cutoff_25772,cutoff_25979,cutoff_26792,cutoff_27119,cutoff_27236,cutoff_27248,cutoff_27690,cutoff_45092
0,11657_5#10,S,S,S,NaN,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
1,11657_5#11,S,S,R,NaN,R,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
2,11657_5#12,S,S,S,NaN,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
3,11657_5#13,S,S,R,NaN,R,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
4,11657_5#14,S,S,R,NaN,S,S,S,S,S,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1931,24742_1#96,S,S,S,NaN,NaN,NaN,S,S,S,...,0,0,0,0,0,0,0,0,0,0
1932,24742_1#97,S,S,S,NaN,NaN,NaN,S,S,S,...,0,0,0,0,0,0,0,0,0,0
1933,24742_1#98,S,S,R,NaN,NaN,NaN,S,S,S,...,0,0,0,0,0,0,0,0,0,0
1934,24742_1#99,S,S,R,NaN,NaN,NaN,S,S,S,...,0,0,0,0,0,0,0,0,0,0


- Observe que o número de linhas corresponde corretamente ao número de isolados, resultando em um total de 1.936 linhas como nos metadados.

- E temos várias colunas que incluem:

     - **1 coluna de número de isolado** com as tags exclusivas para cada um de nossos isolados.
     - **12 rótulos**, um para cada medicamento para o qual tentaremos fazer previsões.
     - **18291 recursos** que usaremos para fazer previsões para os rótulos.
 
Total de colunas: 18304

In [28]:
# Exporta para CSV
df_dsa.to_csv("dataset_final.csv", index = False)

In [29]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [30]:
#%watermark -v -m

In [31]:
#%watermark --iversions

# Fim